In [1]:
import pandas as pd

### Resumo

**Analistas de Dados**
<br> Seu trabalho vai ser entender o perfil de fraude

*júnior*
- Quais métricas (KPIs) são importantes para o problema de fraude? Veja as “notas importantes” para isso e pesquisem!
- Como estão esses nossos KPIs atualmente?
- Quanto estamos perdendo atualmente em fraude x quanto estamos ganhando em aprovação? Note que temos apenas uma amostra de dados, então não temos aqui os valores totais de perdas. Escreva esse disclaimer
- Ententa como está o perfil geral de features - análise univariada. Procure aqui também por anomalias, nulos, etc
- Agora entenda o perfil do fraudador. Veja cada feature em relação aos fraudadores x não fraudadores. Use e abuse de correlações, testes de hipótese, etc
- <s>Faça um PowerBI ou Looker com os principais KPIs e gráficos que geram insights nesse tema</s>

*pleno*
- Use alguns algoritmos para melhor entendimento do perfil de fraude, como regressão logística ou algum outro que achar válido

*sênior*
- Construa “regras duras” para evitarmos fraude sem o uso de machine learning. Aqui são regras mesmo, por exemplo, se valor acima de X então barramos uma transação
- Discuta os prós e contras dessa sua estratégia
- Simule perdas e ganhos da estratégia
- Desenhe um teste AB para testarmos sua estratégia

**Cientistas de Dados**

*júnior*
- Façam uma EDA - Análise exploratória de dados. Podem se basear nas perguntas do analista para isso
- Desenvolva um modelo para predizer a variável Fraude (1-Sim, 0-Não). Quais técnicas testou? Qual selecionou como melhor abordagem? Explique!
- Compare o desempenho do seu algoritmo com o desempenho de um modelo anterior, cujo resultado é encontrado na coluna Score_fraude_modelo do dataset. Quais métricas usou e qual sua conclusão?
- Queremos encontrar o ponto de threshold ideal para o modelo já treinado apresentado na parte (Score, onde 0 (zero) é o risco mais baixo e 100 (cem) o mais alto). Defina o ponto de corte que maximize os lucros da empresa, de forma que todos os pagamentos com menor Score sejam aprovados e os maiores sejam rejeitados.

*pleno*
- Como você pode garantir que o desempenho do modelo no laboratório vai ser um proxy para o desempenho do modelo em produção?
- Supondo que o desempenho produtivo seja muito diferente do esperado, quais você acha que são as causas mais prováveis?
- Se o modelo precisar responder online, no menor tempo possível, o que isso mudaria no desenvolvimento do seu modelo?

<s>*sênior - projeto de alto impacto - ML Engineers*

Para aqueles que estão buscando vagas mais sênior ou realmente um projeto que destaque MUITO você. Minha dica é fazer isso se você já tiver estudado machine learning muito bem e já possa partir para conceitos de deploy 👇
- Use ferramentas como o MLFlow para testes
- Simule um deploy do modelo. Use Docker, Kuberneters, AWS, etc. Tudo que achar necessário para fazer o deploy
- Pense no monitoramento do modelo
</s>

***Notas importante***

- Pra fins desse case os dados são anonimizados propositadamente. Para resolvê-lo, vocês vão precisar fazer algumas suposições. Justifique toda suposição que fizer
- Sabemos que a empresa ganha 10% do valor de um pagamento aprovado corretamente e a cada fraude aprovada perdemos 100% do valor do pagamento.

***Dados***

- **score_1** a **score_10**: São notas de bureau de crédito que a empresa adquiriu para identificar se o comprador é confiável ou outros dados anonimizados que não temos informação sobre
- **País**: pais de compra
- **Produto:** produto que está sendo comprado no e-commerce
- **Categoria_produto**: categoria a qual esse produto se encaixa
- **Entrega_doc_1** a **entrega_doc_3**: Documentos requisitados no momento de fazer a conta. 0 = N = nao entregou. Y = 1 = entregou. Se vazio, considere que nao entregou
- **Score_fraude_modelo**: score dado pelo modelo atual. É a probabilidade daquela compra ser uma fraude ou nao. Quanto mais próximo de 100, maior a certeza do modelo que é fraude
- **Fraude**: informacao se aquela compra era fraudulenta ou nao. Foi inserida após alguns dias, para termos o feedback real se de fato era fraudulenta ou nao. 0 se nao era fraudulenta e 1 se era fraudulenta

### **Cleaning** and **Modeling**

In [ ]:
df = pd.read_csv('../../data/09. TCC/fraude/dados - dados.csv')
cols = list(df.columns)
cols.sort()
df = df[cols].copy()



### **EDA**

In [12]:

print(df.shape)
display(
    pd.DataFrame({
        'Tipo': df.dtypes,
        'Qtde nulos': df.isnull().sum(),
        'Proporção nulos': df.isnull().mean(),
        'Qtde unicos': df.nunique(),
        'Exemplo unicos': df.apply(lambda x: x.unique()[:3])
    })
)

(150000, 20)


,Tipo,Qtde nulos,Proporção nulos,Qtde unicos,Exemplo unicos
categoria_produto,object,0,0.000000,8324,"[cat_8d714cd, cat_64b574b, cat_e9110c5]"
data_compra,object,0,0.000000,145813,"[2020-03-27 11:51:16, 2020-04-15 19:58:08, 202..."
entrega_doc_1,int64,0,0.000000,2,"[1, 0]"
entrega_doc_2,object,108857,0.725713,2,"[nan, Y, N]"
entrega_doc_3,object,0,0.000000,2,"[N, Y]"
fraude,int64,0,0.000000,2,"[0, 1]"
pais,object,194,0.001293,51,"[BR, AR, UY]"
produto,object,0,0.000000,127804,[Máquininha Corta Barba Cabelo Peito Perna Pel...
score_1,int64,0,0.000000,4,"[4, 2, 1]"
score_10,float64,365,0.002433,1793,"[102.0, 0.0, 77.0]"
